TODO:
1) compute expected return based on market risk and estimated risk (fill price based)
2) BUG: risk	market_risk in final resutls seem to be in wrong order implyting something is not being calculated correctly.  must debug this.

In [1]:
# Install yfinance if not already installed
#!pip install yfinance

from IPython.display import display

from tqdm import tqdm 
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def get_historical_returns(ticker, interval=5):
    # Fetch historical data for the stock ticker
    stock_data = yf.download(ticker, start="2000-01-01", end="2024-12-31")
    
    # Calculate the percentage change for each day
    stock_data['Return'] = stock_data['Adj Close'].pct_change()
    
    # Calculate returns over X-day intervals
    stock_data[f'{interval}-Day Return'] = stock_data['Adj Close'].pct_change(periods=interval)
    
    # Drop NaN values that arise from pct_change()
    stock_data.dropna(inplace=True)
    
    return stock_data


def days_until(date_str):
    # Convert the given date string to a datetime object
    target_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Get the current date
    current_date = datetime.today()
    
    # Calculate the difference in days
    difference_in_days = (target_date - current_date).days
    
    return difference_in_days


def get_option_chains_within_45_days(ticker):
    """
    Get the option chains for all expiration dates within the next 45 days for the given ticker.
    """
    # Get the stock ticker object
    stock = yf.Ticker(ticker)
    
    # Get the available expiration dates
    expirations = stock.options
    
    # Calculate the cutoff date for 45 days from now
    cutoff_date = datetime.now() + timedelta(days=45)
    
    # Filter expiration dates within the next 45 days
    filtered_expirations = [exp for exp in expirations if datetime.strptime(exp, '%Y-%m-%d') <= cutoff_date]
    
    if not filtered_expirations:
        #print(f"No expiration dates within the next 45 days for {ticker}.")
        return None
    
    #print(f"Expiration dates within the next 45 days for {ticker}: {filtered_expirations}")
    
    # Get the option chains for all filtered expiration dates
    option_chains = {}
    for expiration in filtered_expirations:
        option_chain = stock.option_chain(expiration)
        option_chains[expiration] = option_chain
    
    return option_chains


def print_option_chain(option_chain, ticker, expiration_date):
    """
    Print the option chain for the given ticker and expiration date.
    """
    print(f"\nOption Chain for {ticker} expiring on {expiration_date}\n")
    print("Calls:")
    display(option_chain.calls.head())
    print("\nPuts:")
    display(option_chain.puts.head())


def get_current_stock_price(ticker):
    """
    Get the current stock price for the given ticker.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period="3mo")
    current_price = history["Close"].iloc[-1]
    sma_10 = history["Close"].tail(10).mean()
    sma_50 = history["Close"].tail(50).mean()
    return current_price, sma_10, sma_50


def dict_to_df(data):
    """
    Convert a dictionary to a pandas DataFrame.
    Ensures that the output is always a DataFrame.
    
    Parameters:
    data (dict): Dictionary to convert.
                 If the dictionary represents a single row, the function will
                 convert it to a DataFrame with one row.
    
    Returns:
    pd.DataFrame: DataFrame representation of the dictionary.
    """
    if isinstance(data, dict):
        # If the dictionary represents a single row, wrap it in a list to ensure one row DataFrame
        df = pd.DataFrame([data]).T
    else:
        # If the data is already in a format suitable for a DataFrame, just convert it
        df = pd.DataFrame(data)
    
    return df


def find_put_spread(puts_f, last_f, margin=.1):

    short_put_candidates = puts_f[puts_f['strike'] <= last_f]
    put_spreads = []
    for _, short_put in short_put_candidates.iterrows():
        long_put_candidates = puts_f[puts_f['strike'] < short_put['strike']]
        if not long_put_candidates.empty:
            long_put = long_put_candidates.iloc[0]  # Get the first available long put

            # Calculate credit and risk
            short_price = (short_put['bid']*.5 + short_put['ask']*.5)
            long_price = (long_put['bid']*.5+long_put['ask']*.5)
            credit = short_price - long_price#short_put['bid'] - long_put['ask']
            market_credit = short_put['bid']-long_put['ask']
            risk = short_put['strike'] - long_put['strike'] - credit
            market_risk = short_put['strike'] - long_put['strike'] - market_credit

            # Check if credit is greater than 50% of the risk   
            if credit > margin * risk:
                put_spreads.append({
                    'short_put_contract': short_put['contractSymbol'],
                    'short_put_strike': short_put['strike'],
                    'short_bid':short_put['bid'],
                    'short_ask':short_put['ask'],
                    'short_price':short_price,
                    'long_put_contract': long_put['contractSymbol'],
                    'long_put_strike': long_put['strike'],
                    'long_bid':long_put['bid'],
                    'long_ask':long_put['ask'],
                    'long_price':long_price,
                    'last_price':last_f,
                    'credit': credit,
                    'market_credit':market_credit,
                    'risk': risk,
                    'market_risk':market_risk,
                    'c/r': credit/risk,
                    'market_c/r':market_credit/market_risk,
                    'ITM_return':(short_put['strike']-last_f)/last_f, 
                    'expected_return':''             
                })

    return pd.DataFrame(put_spreads)

In [3]:
# Get option chains and current stock prices for Google and Apple
MARGIN = 0.1
tickers = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "FB", "TSLA", "BRK.B", "NVDA", "JPM",
    "JNJ", "V", "UNH", "PG", "HD", "BAC", "DIS", "MA", "PYPL", "VZ",
    "ADBE", "NFLX", "INTC", "CMCSA", "PFE", "KO", "PEP", "T", "XOM", "CSCO",
    "ABT", "MRK", "NKE", "CRM", "LLY", "TMO", "AVGO", "WMT", "ACN", "COST",
    "DHR", "NEE", "MCD", "MDT", "WFC", "HON", "UNP", "LIN", "TXN", "AMGN",
    "MS", "PM", "BMY", "QCOM", "SCHW", "C", "IBM", "ORCL", "BA", "UPS",
    "GS", "SBUX", "RTX", "BLK", "CAT", "CVX", "LOW", "SPGI", "AMT", "INTU",
    "MMM", "ISRG", "MDLZ", "GE", "AXP", "PLD", "ADP", "CB", "MO", "ZTS",
    "CCI", "SYK", "TGT", "DE", "BDX", "SCHW", "MMC", "DUK", "CI", "PGR",
    "SO", "USB", "TFC", "EL", "ADI", "ITW", "HUM", "AMAT", "NOC", "MU",
    "BKNG", "GM", "GILD", "FIS", "LMT", "TJX", "NSC", "FISV", "CSX", "HCA",
    "PNC", "ICE", "COP", "MET", "EW", "VRTX", "TRV", "ETN", "CME", "PSA",
    "MCO", "GD", "MPC", "ADSK", "MAR", "KMB", "AON", "FDX", "ADP", "ECL",
    "NOC", "APH", "JCI", "DG", "COF", "SPG", "KLAC", "MNST", "AZO", "APTV",
    "FTNT", "EOG", "ROST", "BAX", "RMD", "CTAS", "KMI", "O", "WMB", "IQV",
    "MCK", "BSX", "D", "CDNS", "AEP", "SHW", "TT", "EA", "PEG", "RSG",
    "STZ", "ATO", "EXC", "CTSH", "PPG", "WY", "DOV", "PSX", "ALL", "WEC",
    "SRE", "CMG", "PCAR", "HSY", "CARR", "ILMN", "AME", "AVB", "SLB", "BKR",
    "ZBH", "MSCI", "PH", "HPQ", "HES", "DTE", "ES", "DLR", "DVN", "OKE",
    "FAST", "AFL", "VFC", "CERN", "MSI", "DHI", "EBAY", "ED", "MTB", "SYY",
    "WAT", "AIG", "RCL", "FMC", "MKC", "LYB", "FTV", "EMN", "EMR", "CMS",
    "CSL", "ATO", "KEYS", "BBY", "HRL", "FLT", "TSCO", "GWW", "XRAY", "LUV",
    "FFIV", "TDY", "XEL", "REGN", "WDC", "TROW", "WYNN", "HIG", "J", "DISH",
    "CAG", "UAL", "A", "AKAM", "PWR", "COO", "MAS", "NRG", "BBY", "ETR",
    "ANET", "PAYC", "DD", "CTXS", "DRE", "SEE", "ZBRA", "ESS", "ODFL", "DPZ",
    "SBAC", "AEE", "ABC", "LNT", "NTAP", "K", "MKTX", "CINF", "MRO", "IR",
    "VRSK", "NDAQ", "HST", "ROK", "ALLE", "KMX", "LH", "EXR", "ANSS", "LDOS",
    "TDG", "TECH", "NRG", "NTAP", "CNP", "HII", "AAP", "WRB", "ALGN", "CBOE",
    "HOLX", "AES", "LHX", "CPT", "CCL", "CMS", "MAS", "MTD", "NLOK", "EXPD",
    "HAS", "GPC", "IP", "OKE", "VMC", "IPG", "PKI", "ETSY", "WST", "AVY",
    "HWM", "XRAY", "CFG", "BEN", "CDW", "NTRS", "LUMN", "PXD", "POOL", "PEAK",
    "RF", "UDR", "CMA", "DRI", "FRT", "AES", "VTRS", "WHR", "PBCT", "JPM",
    "FTI", "TAP", "WAB", "CLX", "FE", "AES", "CMS", "AWK", "PPL", "PNW",
    "ESS", "FITB", "BF.B", "OKE", "DLR", "ATO", "ESS", "CMS", "VTRS", "TAP",
    "EXPE", "ABMD", "ARE", "MAA", "UDR", "FMC", "BRX", "MPWR", "WYNN", "FRT",
    "PPL", "ATO", "CMS", "VTRS", "CPT", "UDR", "HRB", "ZION", "BEN", "WY",
    "HBAN", "PFG", "MTB", "CFG", "GPC", "EVRG", "SLG", "SBAC", "STT", "FRC"
]


In [4]:
put_spreads_df = []
for ticker in tqdm(tickers):
    try:
        option_chains = get_option_chains_within_45_days(ticker)
        exp_dates = list(option_chains.keys())
        for e_d in exp_dates:
            # stock return distribution and compute expected returns
            days = days_until(date_str=e_d)+1  # this should be moved into find put spread function
            if days < 2: continue
            return_dist = get_historical_returns(ticker, interval=days) # this should be moved into find put spread function

            # put trade
            puts = option_chains[e_d].puts
            puts = puts[puts.inTheMoney==False]
            last, sma_10, sma_50 = get_current_stock_price(ticker)
            puts=puts.sort_values(by='strike', ascending=False)
            
            put_spread = find_put_spread(puts_f=puts, last_f=last, margin=MARGIN)

            if not put_spread.empty: # this should be moved into find put spread function
                
                vals = return_dist[f'{days}-Day Return'].values
                p = np.zeros(put_spread.shape[0])
                for idx, itm_val in enumerate(put_spread['ITM_return']):
                    p[idx] = 1-len(vals[vals<itm_val])/len(vals)

                return_break_even = ((put_spread['short_put_strike']-put_spread['credit']) - put_spread['last_price'])/put_spread['last_price']
                p_break_even = np.zeros(put_spread.shape[0])
                for idx, bke_val in enumerate(return_break_even):
                    p_break_even[idx] = 1-len(vals[vals<bke_val])/len(vals)
                    
                put_spread['P(success)']=p
                put_spread['P(gain)']=p_break_even
                put_spread['P(loss)']=1-p_break_even
                put_spread['break_even_return'] = return_break_even
                put_spread["expected_value"] = p*put_spread['credit'] + (p_break_even-p)*put_spread['credit']/2 - (1-p_break_even)*put_spread['risk']
                put_spread["expected_return"] = (put_spread["expected_value"])/put_spread['risk']
                put_spread["expected_annual_return"] = (put_spread["expected_return"])/(days/365)

                put_spread["expected_market_value"] = p*put_spread['market_credit'] - (1-p)*put_spread['market_risk']
                put_spread["expected_market_return"] = (put_spread["expected_market_value"])/put_spread['market_risk']
                put_spread["expected_annual_market_return"] = (put_spread["expected_market_return"])/(days/365)

                put_spread['days']=days
                put_spread['sma_10']=sma_10
                put_spread['sma_50']=sma_50
                put_spreads_df.append(put_spread)
    except:  continue    
    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
return_break_even
put_spread
#1-len(vals[vals<put_spread['ITM_return']])/len(vals)

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,...,P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50
0,STT240920P00085000,85.0,1.2,1.35,1.275,STT240920P00082500,82.5,0.5,0.6,0.55,...,0.367638,-0.014443,-0.209345,-1.871667,-0.424272,-0.223301,-3.543689,23,82.613001,79.153983
1,STT240920P00082500,82.5,0.5,0.60,0.550,STT240920P00080000,80.0,0.2,0.3,0.25,...,0.269741,-0.038709,-0.376367,-2.714903,-0.507929,-0.220839,-3.504613,23,82.613001,79.153983


In [6]:
all_apreads=pd.concat(put_spreads_df)

final=pd.concat(put_spreads_df)
gap = (final.short_put_strike-final.last_price)/final.last_price
final['gap']=gap

final['Credit/SmL'] = final.credit/(final.short_put_strike - final.long_put_strike)
final['date']=datetime.today().date()

select1 = final['ITM_return']>-0.2
select2 = (final['long_ask'] - final['long_bid'])/final['long_ask']<0.4
select3 = (final['short_ask'] - final['short_bid'])/final['short_ask']<0.4
select4 = final['expected_annual_return']>0
final = final[select1 & select2 & select3 & select4]

final = final.sort_values(by='expected_annual_return', ascending=False)


In [7]:
final.to_csv('data/put_spread_trades.csv', index=False)

In [8]:
df = pd.read_csv('data/put_spread_trades_all.csv')
updated_df = pd.concat([df, final], ignore_index=True)
updated_df.to_csv('data/put_spread_trades_all.csv', index=False)

In [9]:
pd.set_option('display.max_columns', None)

final.head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
6,NVDA240830P00120000,120.0,3.50,3.60,3.550,NVDA240830P00119000,119.0,2.00,3.10,2.55,126.968002,1.000,0.40,0.000,0.60,inf,0.666667,-0.054880,inf,0.911950,0.930656,0.069344,-0.062756,0.921303,inf,0.311950,0.519916,94.884696,2,126.802800,120.684360,-0.054880,1.00,2024-08-28
1,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.629997,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.012750,35.227625,0.696178,0.751814,0.248186,-0.018278,0.352276,6429.041485,-0.021911,-0.059219,-10.807444,2,86.641999,85.808400,-0.012750,0.98,2024-08-28
2,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.550003,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.011858,34.722383,0.683277,0.744396,0.255604,-0.017391,0.347224,6336.834986,-0.028362,-0.076653,-13.989145,2,86.634000,85.806800,-0.011858,0.98,2024-08-28
3,AVGO240830P00157500,157.5,2.35,2.70,2.525,AVGO240830P00157000,157.0,1.95,2.15,2.05,160.100006,0.475,0.20,0.025,0.30,19.000000,0.666667,-0.016240,14.794801,0.774083,0.803906,0.196094,-0.019207,0.369870,2700.051135,0.087041,0.290138,52.950207,2,164.110001,160.131689,-0.016240,0.95,2024-08-28
0,BBY240830P00088500,88.5,2.87,3.45,3.160,BBY240830P00088000,88.0,2.57,2.89,2.73,88.629997,0.430,-0.02,0.070,0.52,6.142857,-0.038462,-0.001467,3.117942,0.536526,0.606676,0.393324,-0.006318,0.218256,569.024380,-0.251737,-0.484109,-88.349940,2,86.641999,85.808400,-0.001467,0.86,2024-08-28


In [10]:
def plot_function(df, xcol, ycol,title):
    fig = px.scatter(df, x=xcol, y=ycol, 
        hover_data=['short_put_contract', 'long_put_contract', 'days', 'ITM_return','c/r','P(success)','expected_annual_market_return','last_price','sma_10',	'sma_50'],)
    fig.update_layout(title=title)
    fig.show()

temp = final[final['expected_annual_market_return']>0.2]
plot_function(df=temp, xcol='days', ycol='gap', title="Today")
plot_function(df=updated_df[updated_df['expected_annual_market_return']>0.2], xcol='days', ycol='gap', title="Historical")


In [11]:
pd.set_option('display.max_rows', None)
final.head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
6,NVDA240830P00120000,120.0,3.50,3.60,3.550,NVDA240830P00119000,119.0,2.00,3.10,2.55,126.968002,1.000,0.40,0.000,0.60,inf,0.666667,-0.054880,inf,0.911950,0.930656,0.069344,-0.062756,0.921303,inf,0.311950,0.519916,94.884696,2,126.802800,120.684360,-0.054880,1.00,2024-08-28
1,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.629997,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.012750,35.227625,0.696178,0.751814,0.248186,-0.018278,0.352276,6429.041485,-0.021911,-0.059219,-10.807444,2,86.641999,85.808400,-0.012750,0.98,2024-08-28
2,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.550003,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.011858,34.722383,0.683277,0.744396,0.255604,-0.017391,0.347224,6336.834986,-0.028362,-0.076653,-13.989145,2,86.634000,85.806800,-0.011858,0.98,2024-08-28
3,AVGO240830P00157500,157.5,2.35,2.70,2.525,AVGO240830P00157000,157.0,1.95,2.15,2.05,160.100006,0.475,0.20,0.025,0.30,19.000000,0.666667,-0.016240,14.794801,0.774083,0.803906,0.196094,-0.019207,0.369870,2700.051135,0.087041,0.290138,52.950207,2,164.110001,160.131689,-0.016240,0.95,2024-08-28
0,BBY240830P00088500,88.5,2.87,3.45,3.160,BBY240830P00088000,88.0,2.57,2.89,2.73,88.629997,0.430,-0.02,0.070,0.52,6.142857,-0.038462,-0.001467,3.117942,0.536526,0.606676,0.393324,-0.006318,0.218256,569.024380,-0.251737,-0.484109,-88.349940,2,86.641999,85.808400,-0.001467,0.86,2024-08-28


In [12]:
final.sort_values(by='ITM_return').head()#(#, ascending=True)


,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
23,NVDA240920P00102000,102.0,1.31,1.38,1.345,NVDA240920P00101000,101.0,1.18,1.27,1.225,126.968002,0.12,0.04,0.88,0.96,0.136364,0.041667,-0.196648,0.053023,0.926052,0.926699,0.073301,-0.197593,0.046660,0.841451,-0.033948,-0.035363,-0.561191,23,126.802800,120.684360,-0.196648,0.12,2024-08-28
24,NVDA240927P00102000,102.0,1.56,1.66,1.610,NVDA240927P00101000,101.0,1.44,1.54,1.490,126.968002,0.12,0.02,0.88,0.98,0.136364,0.020408,-0.196648,0.028239,0.903936,0.904908,0.095092,-0.197593,0.024850,0.343572,-0.076064,-0.077616,-0.944326,30,126.802800,120.684360,-0.196648,0.12,2024-08-28
22,NVDA241004P00102000,102.0,1.83,1.97,1.900,NVDA241004P00101000,101.0,1.66,1.82,1.740,126.968002,0.16,0.01,0.84,0.99,0.190476,0.010101,-0.196648,0.053705,0.884366,0.885177,0.114823,-0.197908,0.045112,0.529789,-0.105634,-0.106701,-1.052592,37,126.802800,120.684360,-0.196648,0.16,2024-08-28
18,AVGO240920P00129000,129.0,0.70,0.90,0.800,AVGO240920P00128000,128.0,0.65,0.75,0.700,160.100006,0.10,-0.05,0.90,1.05,0.111111,-0.047619,-0.194254,0.101970,0.991773,0.991773,0.008227,-0.194878,0.091773,1.618216,-0.058227,-0.055454,-0.880038,23,164.110001,160.131689,-0.194254,0.10,2024-08-28
23,NVDA240927P00103000,103.0,1.68,1.78,1.730,NVDA240927P00102000,102.0,1.56,1.66,1.610,126.968002,0.12,0.02,0.88,0.98,0.136364,0.020408,-0.188772,0.019955,0.896647,0.897619,0.102381,-0.189717,0.017560,0.242785,-0.083353,-0.085054,-1.034829,30,126.802800,120.684360,-0.188772,0.12,2024-08-28


In [13]:
final.sort_values(by='expected_annual_return',  ascending=False).head()


,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
6,NVDA240830P00120000,120.0,3.50,3.60,3.550,NVDA240830P00119000,119.0,2.00,3.10,2.55,126.968002,1.000,0.40,0.000,0.60,inf,0.666667,-0.054880,inf,0.911950,0.930656,0.069344,-0.062756,0.921303,inf,0.311950,0.519916,94.884696,2,126.802800,120.684360,-0.054880,1.00,2024-08-28
1,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.629997,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.012750,35.227625,0.696178,0.751814,0.248186,-0.018278,0.352276,6429.041485,-0.021911,-0.059219,-10.807444,2,86.641999,85.808400,-0.012750,0.98,2024-08-28
2,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.28,88.550003,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.011858,34.722383,0.683277,0.744396,0.255604,-0.017391,0.347224,6336.834986,-0.028362,-0.076653,-13.989145,2,86.634000,85.806800,-0.011858,0.98,2024-08-28
3,AVGO240830P00157500,157.5,2.35,2.70,2.525,AVGO240830P00157000,157.0,1.95,2.15,2.05,160.100006,0.475,0.20,0.025,0.30,19.000000,0.666667,-0.016240,14.794801,0.774083,0.803906,0.196094,-0.019207,0.369870,2700.051135,0.087041,0.290138,52.950207,2,164.110001,160.131689,-0.016240,0.95,2024-08-28
0,BBY240830P00088500,88.5,2.87,3.45,3.160,BBY240830P00088000,88.0,2.57,2.89,2.73,88.629997,0.430,-0.02,0.070,0.52,6.142857,-0.038462,-0.001467,3.117942,0.536526,0.606676,0.393324,-0.006318,0.218256,569.024380,-0.251737,-0.484109,-88.349940,2,86.641999,85.808400,-0.001467,0.86,2024-08-28


In [14]:
select1 = final.days<=5
select2 = final.last_price > .97*final.sma_10
select3 = final.sma_10 > final.sma_50
final[select1 & select2 & select3]

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
6,NVDA240830P00120000,120.0,3.50,3.60,3.550,NVDA240830P00119000,119.0,2.00,3.10,2.550,126.968002,1.000,0.40,0.000,0.60,inf,0.666667,-0.054880,inf,0.911950,0.930656,0.069344,-0.062756,0.921303,inf,0.311950,0.519916,94.884696,2,126.802800,120.684360,-0.054880,1.000,2024-08-28
1,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.280,88.629997,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.012750,35.227625,0.696178,0.751814,0.248186,-0.018278,0.352276,6429.041485,-0.021911,-0.059219,-10.807444,2,86.641999,85.808400,-0.012750,0.980,2024-08-28
2,BBY240830P00087500,87.5,2.58,2.96,2.770,BBY240830P00087000,87.0,2.11,2.45,2.280,88.550003,0.490,0.13,0.010,0.37,49.000000,0.351351,-0.011858,34.722383,0.683277,0.744396,0.255604,-0.017391,0.347224,6336.834986,-0.028362,-0.076653,-13.989145,2,86.634000,85.806800,-0.011858,0.980,2024-08-28
3,AVGO240830P00157500,157.5,2.35,2.70,2.525,AVGO240830P00157000,157.0,1.95,2.15,2.050,160.100006,0.475,0.20,0.025,0.30,19.000000,0.666667,-0.016240,14.794801,0.774083,0.803906,0.196094,-0.019207,0.369870,2700.051135,0.087041,0.290138,52.950207,2,164.110001,160.131689,-0.016240,0.950,2024-08-28
0,BBY240830P00088500,88.5,2.87,3.45,3.160,BBY240830P00088000,88.0,2.57,2.89,2.730,88.629997,0.430,-0.02,0.070,0.52,6.142857,-0.038462,-0.001467,3.117942,0.536526,0.606676,0.393324,-0.006318,0.218256,569.024380,-0.251737,-0.484109,-88.349940,2,86.641999,85.808400,-0.001467,0.860,2024-08-28
0,AVGO240830P00160000,160.0,3.40,3.70,3.550,AVGO240830P00159000,159.0,2.70,2.90,2.800,160.100006,0.750,0.50,0.250,0.50,3.000000,1.000000,-0.000625,1.430852,0.562681,0.634732,0.365268,-0.005309,0.357713,261.130575,0.062681,0.125363,22.878728,2,164.110001,160.131689,-0.000625,0.750,2024-08-28
4,NVDA240830P00122000,122.0,4.20,4.30,4.250,NVDA240830P00121000,121.0,3.60,3.70,3.650,126.968002,0.600,0.50,0.400,0.50,1.500000,1.000000,-0.039128,1.167594,0.851798,0.873569,0.126431,-0.043854,0.467038,213.085893,0.351798,0.703596,128.406305,2,126.802800,120.684360,-0.039128,0.600,2024-08-28
3,BBY240830P00086000,86.0,1.83,2.39,2.110,BBY240830P00085000,85.0,1.48,1.73,1.605,88.629997,0.505,0.10,0.495,0.90,1.020202,0.111111,-0.029674,0.767400,0.851959,0.882600,0.117400,-0.035372,0.379863,140.050472,-0.048041,-0.053378,-9.741574,2,86.641999,85.808400,-0.029674,0.505,2024-08-28
4,BBY240830P00086000,86.0,1.83,2.39,2.110,BBY240830P00085000,85.0,1.48,1.73,1.605,88.550003,0.505,0.10,0.495,0.90,1.020202,0.111111,-0.028797,0.758347,0.845670,0.878729,0.121271,-0.034500,0.375382,138.398338,-0.054330,-0.060367,-11.016906,2,86.634000,85.806800,-0.028797,0.505,2024-08-28
0,BBY240830P00088500,88.5,2.87,3.45,3.160,BBY240830P00088000,88.0,2.73,2.92,2.825,88.550003,0.335,-0.05,0.165,0.55,2.030303,-0.090909,-0.000565,0.697219,0.521690,0.579423,0.420577,-0.004348,0.115041,127.242454,-0.289155,-0.525736,-95.946878,2,86.634000,85.806800,-0.000565,0.670,2024-08-28


In [15]:
all_apreads.tail()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P(success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50
1,SBAC240920P00210000,210.0,0.05,2.70,1.375,SBAC240920P00200000,200.0,0.0,0.60,0.300,228.539993,1.075,-0.55,8.925,10.55,0.120448,-0.052133,-0.081124,-0.038630,0.851133,0.858414,0.141586,-0.085827,-0.344771,-0.613039,-2.038673,-0.193239,-3.066621,23,221.540887,209.140280
2,SBAC240920P00180000,180.0,0.00,2.15,1.075,SBAC240920P00175000,175.0,0.0,0.15,0.075,228.539993,1.000,-0.15,4.000,5.15,0.250000,-0.029126,-0.212392,0.196420,0.955825,0.957282,0.042718,-0.216767,0.785680,3.117098,-0.370874,-0.072014,-1.142836,23,221.540887,209.140280
3,SBAC240920P00160000,160.0,0.00,1.65,0.825,SBAC240920P00155000,155.0,0.0,0.00,0.000,228.539993,0.825,0.00,4.175,5.00,0.197605,0.000000,-0.299904,0.163710,0.970065,0.971845,0.028155,-0.303514,0.683489,2.598006,-0.149676,-0.029935,-0.475060,23,221.540887,209.140280
0,STT240920P00085000,85.0,1.20,1.35,1.275,STT240920P00082500,82.5,0.5,0.60,0.550,85.510002,0.725,0.60,1.775,1.90,0.408451,0.315789,-0.005964,-0.117941,0.590291,0.632362,0.367638,-0.014443,-0.209345,-1.871667,-0.424272,-0.223301,-3.543689,23,82.613001,79.153983
1,STT240920P00082500,82.5,0.50,0.60,0.550,STT240920P00080000,80.0,0.2,0.30,0.250,85.510002,0.300,0.20,2.200,2.30,0.136364,0.086957,-0.035201,-0.171076,0.716828,0.730259,0.269741,-0.038709,-0.376367,-2.714903,-0.507929,-0.220839,-3.504613,23,82.613001,79.153983


In [16]:
plot_function(df=all_apreads[all_apreads.short_bid>0], xcol='days', ycol='ITM_return', title="Today")
